## Create Offspring Branch

In [1]:
import os 
os.chdir('..')

In [2]:
from utils.rpc.lib import json_rpc_call
from utils.encode.bytes import encode_bytes_to_base64_str, decode_base64_str_to_bytes
from config.branch_cfg import TWIG_BRANCH_TYPE_ID
from config.bucket_cfg import BUCKET_ID_TYPE_NO_REF, BUCKET_ID_TYPE_WITH_ID_REF, DEFAULT_MOLECULAR_BUCKET_SCHEMA, DEFAULT_ATOMIC_BUCKET_SCHEMA
BYTES0 = encode_bytes_to_base64_str(bytes(0))

#### Create Genesis Branch

In [3]:
signature = BYTES0
accept_conflicts = False
msg = 'Genesis Submit'
name = 'Genesis Branch'
create_branch_kwargs = dict(branch_type=TWIG_BRANCH_TYPE_ID, name=name, signature=signature, accept_conflicts=accept_conflicts, msg=msg)

In [4]:
genesis_response = json_rpc_call(method="create_genesis_branch", params=create_branch_kwargs)
# response is inside the result field
print("Test 'create_genesis_branch' passed with branch ID:", genesis_response['result'], "\nand base64 decoded branch ID:", decode_base64_str_to_bytes(genesis_response['result']))

Test 'create_genesis_branch' passed with branch ID: AVESCHbLIpIZhFx9 
and base64 decoded branch ID: b'\x01Q\x12\x08v\xcb"\x92\x19\x84\\}'


#### Create Offspring Branch

In [5]:
create_offspring_branch_kwargs = dict(parent_branch_id=genesis_response['result'], **create_branch_kwargs)
create_offspring_branch_kwargs.update(dict(name='Offspring Branch', msg='Offspring Submit'))

In [6]:
offspring_response = json_rpc_call(method="create_offspring_branch_at_head", params=create_offspring_branch_kwargs)
# response is inside the result field
print("Test 'create_offspring_branch_at_head' passed with branch ID:", offspring_response['result'], "\nand base64 decoded branch ID:", decode_base64_str_to_bytes(offspring_response['result']))

Test 'create_offspring_branch_at_head' passed with branch ID: AVESCHBPR/awevnZ 
and base64 decoded branch ID: b'\x01Q\x12\x08pOG\xf6\xb0z\xf9\xd9'


#### Create Offspring branch at particular submit

In [7]:
rpc_kwargs = dict(branch_id=offspring_response["result"], deserialize_buckets=False) #rpc_genesis_branch_response["decoded_branch_id"], deserialize_buckets=True)
response = json_rpc_call(method="get_branch_data_from_branch_id", params=rpc_kwargs)

In [10]:
genesis_submit_id = response["result"]["submit"]["parent_submit_id"] 
response["result"]

{'id': 'AVESCHBPR/awevnZ',
 'ns': 'CLF/HHC3',
 'name': 'Offspring Branch',
 'parent_id': 'AVESCHbLIpIZhFx9',
 'config': {'accept_conflicts': False, 'branch_type': 1},
 'sprouts': [],
 'sprout_selection': [],
 'name_resolution': 'AVESCDSNHSSjvABhEAYIsX8ccLc=',
 'interaction': 'AVESCDSNHSSjvABhEQYIsX8ccLc=',
 'parent_name_resolution': 'AVESCDSNHSSjvABhEQYccLcAAAA=',
 'parent_interaction': 'AVESCDSNHSSjvABhEAYccLcAAAA=',
 'parent_data_trie': 'AVESCDSNHSSjvABhCwYccLcAAAA=',
 'signature': '',
 'creation_ts': 1708121148,
 'submit': {'stable_head_id': 'AVESCPH+h7IbDz40DQYIsX8ccLc=',
  'parent_submit_id': 'AVESCASBOS0KjvidDQYccLcAAAA=',
  'submit_msg': 'Offspring Submit'},
 'submit_trace': {'new_buckets': [], 'new_registered_names': []}}

In [11]:
create_another_offspring_branch_kwargs = dict(
    parent_branch_id=genesis_response['result'],
    parent_submit_id=genesis_submit_id,
    branch_type=TWIG_BRANCH_TYPE_ID,
    name='Another Offspring Branch',
    signature=signature,
    accept_conflicts=accept_conflicts,
    msg='Another Offspring Submit')

In [12]:
another_offspring_response = json_rpc_call(method="create_offspring_branch_at_head", params=create_offspring_branch_kwargs)
# response is inside the result field
print("Test 'create_offspring_branch_at_head' passed with branch ID:", another_offspring_response['result'], "\nand base64 decoded branch ID:", decode_base64_str_to_bytes(another_offspring_response['result']))

Test 'create_offspring_branch_at_head' passed with branch ID: AVESCCMz2VCxLXVU 
and base64 decoded branch ID: b'\x01Q\x12\x08#3\xd9P\xb1-uT'


In [ ]:
contents = [
        {   
            ## A CONTENT BUCKET FOR AN ARTICLE
            "data": "=Hello=",  
            "schema": DEFAULT_ATOMIC_BUCKET_SCHEMA, 
            "parent_id": BYTES0, 
            "signature": BYTES0, 
            "refs": []
        },
        {
            ## ANOTHER CONTENT BUCKET FOR AN ARTICLE
            "data": "=World=",  
            "schema": DEFAULT_ATOMIC_BUCKET_SCHEMA, 
            "parent_id": BYTES0, 
            "signature": BYTES0, 
            "refs": []
        },
        {
            ## A CONTEXT BUCKET (MOLECULAR) FOR THAT ARTICLE
            "data": {
                "order": [
                    {"id": 0, "type": BUCKET_ID_TYPE_NO_REF},
                    {"id": 1, "type": BUCKET_ID_TYPE_NO_REF}], 
                "name": "Dummy Article Name"},
            "schema": DEFAULT_MOLECULAR_BUCKET_SCHEMA,
            "parent_id": BYTES0, 
            "signature": BYTES0, 
            "refs": []
        }
    ]
## NOTE: The order of the contents is important. 